In [241]:
import hashlib

from requests import get

import chainparams
import time
import socket
import struct

In [243]:
LOCAL_WAN_IP = get('https://api.ipify.org').text
PEER_IP = "77.98.116.8"
PEER_PORT = 8333
print(LOCAL_WAN_IP)

77.98.116.8


In [244]:
class Serializable:

    @staticmethod
    def _to_bytes(msg, length=None, byteorder='little'):
        if isinstance(msg, int):    # or isinstance(msg, bool):
            if length == None:
                length = msg.bit_length()
            return msg.to_bytes(length, byteorder)
        elif isinstance(msg, str):
            return msg.encode(encoding='UTF-8', errors='strict')
        elif isinstance(msg, bytes):
            return msg
        else:
            return print("message of type %s not supported by _to_bytes()" % type(msg))

    @staticmethod
    def pad_msg_end(msg, length):
        len_diff = length - len(msg)
        msg += b"\x00" * len_diff
        return msg

In [245]:
magic = chainparams.mainParams.StartString.to_bytes(4, byteorder='big')
print(magic)
type(magic)

b'\xf9\xbe\xb4\xd9'


bytes

In [258]:
class Message(Serializable):

    def __init__(self, command, payload):
        self.magic = chainparams.mainParams.StartString.to_bytes(4, byteorder='little')
        self.command = command
        self.command_bytes = None
        self.length = len(payload).to_bytes(length=4, byteorder='little')
        self.payload = payload
        self.checksum = None
        self.header = None

    def serialize_payload(self):
        if not isinstance(self.payload, bytes):
            self.payload = Serializable._to_bytes(self.payload)
            self.length = len(self.payload).to_bytes(length=4, byteorder='little')
            self.checksum = hashlib.sha256(hashlib.sha256(self.payload).digest()).digest()[:4]

    def generate_header(self):
        self.serialize_payload()
        self.command_bytes = Serializable.pad_msg_end(Serializable._to_bytes(self.command), 12)
        self.length = Serializable.pad_msg_end(Serializable._to_bytes(self.length), 4)
        self.checksum = hashlib.sha256(hashlib.sha256(self.payload).digest()).digest()[:4]

        self.header = b''.join([self.magic, self.command_bytes, self.length, self.checksum])
        return self.header

    def to_bytes(self):
        self.generate_header()
        msg = b''.join([self.header, self.payload])
        return msg
    
    @staticmethod
    def to_var_int(x):
        if x < 0xfd:
            return Serializable._to_bytes(x, length=1)
        elif x <= 0x10000:
            return b"\xfd" + Serializable._to_bytes(x, length=3)
        elif x <= 0x100000000:
            return b"\xfe" + Serializable._to_bytes(x, length=5)
        elif x <= 0x10000000000000000:
            return b"\xff" + Serializable._to_bytes(x, length=9)
        else:
            raise RuntimeError("integer too large for type<var_int>")
        
    @staticmethod
    def to_var_str(x):
        s = Serializable._to_bytes(x)
        l = len(s)
        return Message.to_var_int(l) + s
    
    @staticmethod
    def generate_network_address(ip, port):
        t = struct.pack(b"<q", int(time.time()))
        print(t)
        s = Serializable._to_bytes('services')
        print(s)
        p = Serializable._to_bytes(port, byteorder='big')
        print(p)
        
        if ':' in ip:
            addr = bytes(map(int, ip.split(':')))
        else:
            addr = (b"\x00" * 10) + (b"\xFF" * 2)
            addr_bytes = bytes(map(int, ip.split('.')))
            addr += addr_bytes 
        
        print(addr)
        
        return b"".join([t, s, addr, p])


In [259]:
a = Message('block', '00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d')

In [260]:
print(a.magic)
print(a.command)
print(a.length)
print(a.payload)
print("payload is type %s" % type(a.payload))
print(a.checksum)

b'\xd9\xb4\xbe\xf9'
block
b'@\x00\x00\x00'
00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d
payload is type <class 'str'>
None


In [261]:
a.generate_header()

b'\xd9\xb4\xbe\xf9block\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x88\x8b\xa5&'

In [262]:
a.to_bytes()

b'\xd9\xb4\xbe\xf9block\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x88\x8b\xa5&00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d'

In [263]:
print(a.magic)
print(a.command_bytes)
print(a.length)
print(a.payload)
print("payload is type %s" % type(a.payload))
print(a.checksum)

b'\xd9\xb4\xbe\xf9'
b'block\x00\x00\x00\x00\x00\x00\x00'
b'@\x00\x00\x00'
b'00000000000000001e8d6829a8a21adc5d38d0a473b144b6765798e61f98bd1d'
payload is type <class 'bytes'>
b'\x88\x8b\xa5&'


In [264]:
class Test(Message):
    test_number = 252
    test_message = 'hello, world'
    print(test_number)
    print(Message.to_var_int(test_number))
    print(test_message)
    print(Message.to_var_str(test_message))
    


252
b'\xfc'
hello, world
b'\x0chello, world'


In [265]:
a.generate_network_address("77.98.116.8", 8333)


b'B\xd07\\\x00\x00\x00\x00'
b'services'
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x8d'
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffMbt\x08'


b'B\xd07\\\x00\x00\x00\x00services\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xffMbt\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x8d'

In [289]:
print(struct.pack('i', 4))
print(struct.pack('>i', 4))
text = b'Hello, world'
len_txt = len(text)
print(struct.pack('%ds' % len_txt, b'hello, world'))

b'\x04\x00\x00\x00'
b'\x00\x00\x00\x04'
b'hello, world'


In [290]:
"hello, what's my name again".encode('utf-8')

b"hello, what's my name again"